In [203]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from arch import arch_model
from pytorch_lightning.loggers import  TensorBoardLogger
from scipy.optimize import root
import matplotlib.pyplot as plt
import os 
from datetime import datetime
import json

In [204]:

# Генерация данных
def generate_ground_garch(omega, d, phi, beta, n=3000):
    am = arch_model(None, mean='Zero', vol='FIGARCH', p=1, q=1, power=2)
    params = np.array([omega, d, phi, beta])
    am_data = am.simulate(params, n)
    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()


In [205]:

# Dataset для полных последовательностей
class FullSequenceDataset(Dataset):
    def __init__(self, eps, vol, truncation_size, scale=100):
        super().__init__()
        self.eps_squared = torch.tensor(np.square(eps) * scale)
        self.vol = (torch.tensor(vol * scale))
        self.truncation_size = truncation_size
        self.eps_squared = self.eps_squared.float()
        self.vol = self.vol.float()

    def __len__(self):
        return len(self.eps_squared) - self.truncation_size  # Количество возможных окон

    def __getitem__(self, idx):
        # Возвращаем окно остатков и соответствующий таргет
        return (
            self.eps_squared[idx:idx+self.truncation_size],
            self.eps_squared[idx+self.truncation_size]
        )


In [206]:
class FIGARCHDM(pl.LightningDataModule):
    def __init__(self, eps, vol, truncation_size, batch_size):
        super().__init__()

        self.eps = eps
        self.vol = vol
        self.truncation_size = truncation_size
        self.batch_size = batch_size
    def setup(self, stage = None):
        self.train_dataset = FullSequenceDataset(self.eps, self.vol, self.truncation_size)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size= self.batch_size ,shuffle=False)

In [207]:
class CorrectedNLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred_var, target_eps_squared):

        loss =  (0.5*(torch.log(pred_var)) + (target_eps_squared/(2*pred_var))) 
        return loss.mean()

In [208]:

# Модель FIGARCH с CNN
class FullConvFIGARCH(pl.LightningModule):
    def __init__(self, truncation_size, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.truncation_size = truncation_size
        self.lr = lr
        
        self.weights = nn.Parameter(torch.linspace(1.0, 0.1, truncation_size))
        
        self.loss_fn = CorrectedNLoss()

    def forward(self, x):
        weights = self.weights
        #weights = torch.nn.functional.softplus(self.weights)
        return torch.sum(x * weights, dim=1)

    def training_step(self, batch, batch_idx):
        eps_window, target_eps = batch
        pred_var = self.forward(eps_window)
        loss = self.loss_fn(pred_var, target_eps)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            mode='min', 
            factor=0.5, 
            patience=20, 
            verbose=True
        )
        return {
            'optimizer': optimizer,
            #'lr_scheduler': {
            #    'scheduler': scheduler,
            #     'monitor': 'train_loss'
            #}
        }



In [209]:
# omega > 0 <- 1
# 0 <= d <= 1 <- 2
# 0 <= phi <= (1 - d) / 2 <- 2
# 0 <= beta <= d + phi <- 2


omega, d, phi, beta = 0.1, 0.5, 0.2, 0.3
data, volat = generate_ground_garch(omega, d, phi, beta)


In [210]:
ground_truth = [omega, d, phi, beta]

In [211]:
truncation_size = 5
batch_size = (len(data) - truncation_size)//1
#batch_size = 500

In [212]:
model = FullConvFIGARCH(truncation_size)
dm = FIGARCHDM(data, volat, truncation_size, batch_size)


In [213]:
logger = TensorBoardLogger('tb_logs', 'figarch_model')

In [214]:
trainer = pl.Trainer(max_epochs=2000, accelerator='auto', logger= logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [215]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Arseny\AppData\Roaming\Python\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name         | Type           | Params | Mode 
--------------------------------------------------------
0 | loss_fn      | CorrectedNLoss | 0      | train
  | other params | n/a            | 5      | n/a  
--------------------------------------------------------
5         Trainable params
0         Non-trainable params
5         Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
C:\Users\Arseny\AppData\Roaming\Python\Python312\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` 

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2000` reached.


In [216]:
with torch.no_grad():
    actual_lambdas =  model.weights.data.flip(0)
    print("Learned weights:", actual_lambdas )

Learned weights: tensor([0.7038, 0.3413, 0.2951, 0.1969, 0.1689])


In [229]:
def decompute_lambdas(lambdas):
    from scipy.optimize import least_squares
    
    def equations(vars):
        d, phi, beta = vars
        eqs = [
            phi - beta + d - lambdas[0],
            (d - beta)*(beta - phi) + d*(1 - d)/2 - lambdas[1],
            beta*(d*beta - d*phi - beta**2 + beta*phi + d*(1 - d)/2) + d*(1 - d)/2*((2 - d)/3 - phi) - lambdas[2]
        ]
        return eqs
    
    # Ограничения: 0 ≤ d ≤ 1, 0 ≤ phi ≤ (1-d)/2, 0 ≤ beta ≤ d + phi
    bounds = ([0, 0, 0], [1, 1, 1])
    
    # Начальное приближение (можно попробовать несколько вариантов)
    initial_guess = [0.5, 0.1, 0.3]
    
    sol = least_squares(equations, initial_guess)
    
    if not sol.success:
        print("Warning: Solution not found!", sol.message)
    
    return sol.x

In [230]:
model_outputs =decompute_lambdas(actual_lambdas)

#Думаю нужно разобраться с выходами модели - возможно несоответствие по индексам

In [231]:
model_outputs

array([ 1.41139252, -0.1403552 ,  0.63007106])

In [232]:
len(model_outputs)

3

In [233]:
model.weights.data

tensor([0.1689, 0.1969, 0.2951, 0.3413, 0.7038])

In [234]:
def compute_omega(weights, eps_squared, vol_series):
    """
    weights: learned lambdas (λ)
    eps_squared: εₜ² (квадраты остатков)
    vol_series: σₜ² (волатильность в квадрате)
    """
    weights = weights.numpy()
    truncation_size = len(weights)
    
    # Предсказание без omega: Σλᵢεₜ₋ᵢ²
    pred = np.sum(weights * eps_squared[:truncation_size])
    
    # omega = σₜ² - Σλᵢεₜ₋ᵢ²
    omega = vol_series[truncation_size] - pred
    
    return max(omega, 1e-6)  # Омега должна быть строго положительной

In [235]:
pred_omega = compute_omega(actual_lambdas, data, volat)

In [236]:
pred_omega

1.567960819868222

In [237]:
def save_results(model_outputs, ground_truth, pred_omega, filename='figarch_results.json'):
    # Подготовка данных
    result = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'ground_truth': {
            'omega' : float(omega),
            'd': float(ground_truth[1]),
            'phi': float(ground_truth[2]),
            'beta': float(ground_truth[3]),
        },
        'model_params': {
            'omega' : float(pred_omega),
            'd': float(model_outputs[0]),
            'phi': float(model_outputs[1]),
            'beta': float(model_outputs[2])
        }

    }
    
    # Запись в файл (дозапись)
    mode = 'a' if os.path.exists(filename) else 'w'
    with open(filename, mode, encoding='utf-8') as f:
        f.write(json.dumps(result, indent=4) + '\n')  # Добавляем перевод строки

In [238]:
save_results(model_outputs, ground_truth, pred_omega)